# 와인분류기 튜닝하기
* 2020.02.17.mon pm 2:59

신경망... 구현하다보니깐 뭔가 신기하다...!<br/>
노드의 수에 따라, 레이어의 개수에 따라... <br/>
학습률, 어떤 optimizer를 사용하느냐에 따라...! 결과가 조금씩 다르다!<br/>


손글씨 분류기를 만들기 전, 레이어를 한 번 다르게 쌓아보자.<br/>
노드의 개수도 바꿔보고, 레이어의 수도 바꿔보고!

## 학습 프로세스
* 개요
    * 1. 학습 데이터 준비
    * 2. 데이터 전처리
        * 설명변수, 목적변수 분할
        * train_test_split()
    * 3. 텐서 생성
        * 훈련 데이터와 테스트 데이터 텐서 변환
        * 설명변수와 목적변수 병합
        * Mini-Batch
    * 4. 신경망 구성
    * 5. 모형 학습
    * 6. Accuracy test

# #1. 학습 데이터 준비

* Import Libaray
* Load Data
* EDA

### Import Library

In [1]:
# Import Python Library

import torch
from torch.autograd import Variable  # Tensor를 래핑하고 계산과정을 기록하는 역할을 한다.
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Import Scikit-learn Library
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# import pandas library
import pandas as pd

### Load Data

In [2]:
wine = load_wine()

# Pandas DataFrame로 데이터 출력
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


178x13 데이터 프레임

### EDA

In [3]:
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
wine.data

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [5]:
wine.data.shape

(178, 13)

In [6]:
wine.feature_names

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [7]:
len(wine.feature_names)

13

In [8]:
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [9]:
len(wine.target)  # 178

178

# #2. 데이터 전처리

* 설명변수, 목적변수 분할
* train_test_split

### 설명변수, 목적변수 분할

In [10]:
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [11]:
import numpy as np

# wine_target unique값 출력하기
# NumPy Library의 unique()함수 사용

np.unique(wine_target)

array([0, 1])

### train_test_split 

In [12]:
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.2)
print('train_X.shape:', train_X.shape)
print('train_Y.shape:', train_Y.shape)
print('test_X.shape:', test_X.shape)
print('test_X.shape:', test_Y.shape)


train_X.shape: (104, 13)
train_Y.shape: (104,)
test_X.shape: (26, 13)
test_X.shape: (26,)


# #3. 텐서 생성

* 훈련데이터와 테스트 데이터 텐서 변환
* 설명변수와 목적변수 병합
* Mini-Batch

### 훈련 데이터와 테스트 데이터 텐서변환

훈련 데이터와 테스트 데이터를 텐서로 변환해준다

In [13]:
# 훈련 데이터 텐서 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 변환
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터의 형상을 확인해보자
print('train_X.shape', train_X.shape)
print('train_Y.shape', train_Y.shape)
print('test_X.shape', test_X.shape)
print('test_Y.shape', test_Y.shape)

train_X.shape torch.Size([104, 13])
train_Y.shape torch.Size([104])
test_X.shape torch.Size([26, 13])
test_Y.shape torch.Size([26])


### 설명변수와 목적변수 병합

In [14]:
train = TensorDataset(train_X, train_Y)
train[0]

(tensor([1.3770e+01, 1.9000e+00, 2.6800e+00, 1.7100e+01, 1.1500e+02, 3.0000e+00,
         2.7900e+00, 3.9000e-01, 1.6800e+00, 6.3000e+00, 1.1300e+00, 2.9300e+00,
         1.3750e+03]), tensor(0))

### Mini-Batch

batch size가 16인 미니배치로 분할

In [15]:
train_loader = DataLoader(train, batch_size=16, shuffle=True)

# #4.신경망 구성

학습에 사용할 다층퍼셉트론 구성<br/>
기존 5층 신경망을 뛰어넘는 신경망을 구성해보자!

In [38]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):  # torch.nn.Module: 모든 신경망 모듈의 기본이 되는 클래스다
                       # 이 클래스 안에 각 층과 함수, 신경망의 구조를 정의한다
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x)
    
# 인스턴스 생성
model = Net()

# #5. 모형 학습

앞서 #4. 신경망 구성에서 생성한 신경망을 학습한다

* 오차함수 객체 생성
* 최적화 객체 생성
* 학습 시작
* 100 epoch마다 오차 출력

In [21]:
# 오차함수 객체
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    # 분할해온 데이터를 꺼내온다.
    for train_x, train_y in train_loader:
        # 계산그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)  # output라는 인스턴스 선언 하나만으로 순전파 끝!
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()  # 오차 계산하고 backward()만 하면 역전파가 계산된다!
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    # 100회마다 누적 오차 출력
    if (epoch+1) % 100 == 0:
        print(epoch+1,'Epoch: ', total_loss)


C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


100 Epoch:  tensor(1.5436)
200 Epoch:  tensor(1.3846)
300 Epoch:  tensor(1.1881)


# #6. Accuracy Test 

학습된 모형의 정확도를 평가해본다

In [22]:
# 계산 그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9230769230769231

## Test1. 80% -> 50% 성능 저하 모델
* 3층 레이어 사용(노드의 개수는 128개)

**결과**<br/>
학습   오차 4.8149<br/>
정답률 0.5384615... 

## Test2. Test1에서 optimizer를 SGD에서 Adam으로 바꿔주면 어떤결과가 나올까?

In [23]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):  # torch.nn.Module: 모든 신경망 모듈의 기본이 되는 클래스다
                       # 이 클래스 안에 각 층과 함수, 신경망의 구조를 정의한다
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x)
    
# 인스턴스 생성
model = Net()

# 오차함수 객체
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
# optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    # 분할해온 데이터를 꺼내온다.
    for train_x, train_y in train_loader:
        # 계산그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)  # output라는 인스턴스 선언 하나만으로 순전파 끝!
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()  # 오차 계산하고 backward()만 하면 역전파가 계산된다!
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    # 100회마다 누적 오차 출력
    if (epoch+1) % 100 == 0:
        print(epoch+1,'Epoch: ', total_loss)


C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


100 Epoch:  tensor(1.3405)
200 Epoch:  tensor(0.9957)
300 Epoch:  tensor(1.2892)


In [24]:
# Accuracy test

# 계산그래프 구성
test_x, test_y = Variable(test_x), Variable(test_y)

# 출력이 0 또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.8846153846153846

## Test3. 레이어의 수를 늘려보자!
* 레이어별 노드수는 128개로 동일
* 3개의 레이러를 7개까지 늘려보자

In [25]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 128)
        self.fc7 = nn.Linear(128, 128)
        self.fc8 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.fc8(x)
        return F.log_softmax(x)

# 인스턴스 생성
model = Net()

# 오차 함수 객체
criterion = nn.CrossEntropyLoss()

# 최적화 객체 생성
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(300):
    total_loss = 0
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계싼
        total_loss += loss.data
        
    # 100회마다 누적 오차 출력
    if (epoch+1) % 100 == 0:
        print( 'Epoch:', epoch+1,'Total_Loss: ', total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 100 Total_Loss:  tensor(1.7526)
Epoch: 200 Total_Loss:  tensor(1.5223)
Epoch: 300 Total_Loss:  tensor(1.4318)


In [26]:
# Accuracy test

# 계산그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9615384615384616

헉...!

### Optimizer를 Adam으로 바꿔보자!

In [29]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 128)
        self.fc7 = nn.Linear(128, 128)
        self.fc8 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.fc8(x)
        return F.log_softmax(x)

# 인스턴스 생성
model = Net()

# 오차 함수 객체
criterion = nn.CrossEntropyLoss()

# 최적화 객체 생성
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 학습 시작
for epoch in range(2000):
    total_loss = 0
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계싼
        total_loss += loss.data
        
    # 100회마다 누적 오차 출력
    if (epoch+1) % 100 == 0:
        print( 'Epoch:', epoch+1,'Total_Loss: ', total_loss)

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 100 Total_Loss:  tensor(1.1575)
Epoch: 200 Total_Loss:  tensor(1.3295)
Epoch: 300 Total_Loss:  tensor(1.0866)
Epoch: 400 Total_Loss:  tensor(1.1427)
Epoch: 500 Total_Loss:  tensor(0.9653)
Epoch: 600 Total_Loss:  tensor(0.8120)
Epoch: 700 Total_Loss:  tensor(0.4187)
Epoch: 800 Total_Loss:  tensor(0.6151)
Epoch: 900 Total_Loss:  tensor(1.3309)
Epoch: 1000 Total_Loss:  tensor(0.4407)
Epoch: 1100 Total_Loss:  tensor(0.6872)
Epoch: 1200 Total_Loss:  tensor(0.4723)
Epoch: 1300 Total_Loss:  tensor(0.5036)
Epoch: 1400 Total_Loss:  tensor(0.3681)
Epoch: 1500 Total_Loss:  tensor(0.3222)
Epoch: 1600 Total_Loss:  tensor(0.2897)
Epoch: 1700 Total_Loss:  tensor(0.8914)
Epoch: 1800 Total_Loss:  tensor(0.5136)
Epoch: 1900 Total_Loss:  tensor(0.6550)
Epoch: 2000 Total_Loss:  tensor(0.2092)


In [30]:
# Accuracy test

# 계산그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 또는 1이 되도록 변환
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 측정
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 모형의 정확도 출력
accuracy

C:\Users\stevelee\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


1.0

Accuracy가 1이라니...;;<br/>
이것저것 시도해봐서 재미있는데... 원리를 좀 더 파악하고 싶은 욕심이...ㅎㅎ<br/>
그리고... GPU를 안쓰고... CPU를 써서 학습하네...<br/>
20.02.17.mon pm4:12